### Homework
Ваша задача спарсить информацию о компаниях, находящихся в индексе S&P 500 с данного сайта: <br>
https://markets.businessinsider.com/index/components/s&p_500

Для каждой компании собрать следующую информацию:
* Текущая стоимость в рублях (конвертацию производить по текущему курсу, взятому с сайта [центробанка РФ](http://www.cbr.ru/development/sxml/))
* Код компании (справа от названия компании на странице компании)
* P/E компании (информация находится справа от графика на странице компании)
* Годовой рост/падение компании в процентах (основная таблица)
* Высчитать какую прибыль принесли бы акции компании (в процентах), если бы они были куплены на уровне 52 Week Low и проданы на уровне 52 Week High (справа от графика на странице компании)

Сохранить итоговую информацию в 4 JSON файла:
1. Топ 10 компаний с самими дорогими акциями в рублях.
2. Топ 10 компаний с самым низким показателем P/E.
3. Топ 10 компаний, которые показали самый высокий рост за последний год
4. Топ 10 комппаний, которые принесли бы наибольшую прибыль, если бы были куплены на самом минимуме и проданы на самом максимуме за последний год.
<br>Пример формата:
```
[
{
    "code": "MMM",
    "name": "3M CO.",
    "price" | "P/E" | "growth" | "potential profit" : value,
},
...
]
```
For scrapping you cans use `beautifulsoup4` <br>
For requesting `aiohttp`

In [168]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from bs4 import element
import requests
import xml.etree.ElementTree as ET
import asyncio
import aiohttp
import time
import re

In [132]:
#получаем ссылки на все компании в списке и, заодно, годовой рост/падение компании
company_links = []
growth = []
for i in range(1,11):
    url = ('https://markets.businessinsider.com/index/components/s&p_500?p='+str(i))
    response = requests.get(url)
    bs = BeautifulSoup(response.text, "html.parser")
    
    for company_href in bs.find_all(name = "td", attrs = {"class": "table__td table__td--big"}):
        link = company_href.find(name = "a", href = True).get('href')
        company_links.append('https://markets.businessinsider.com'+link)
    
    tables = bs.find_all(name = "tr")
    for table in tables[1:]:
        columns = table.find_all(name = "td")
        growth.append(float(columns[7].text.split()[-1][:-1]))
company_links

['https://markets.businessinsider.com/stocks/mmm-stock',
 'https://markets.businessinsider.com/stocks/aos-stock',
 'https://markets.businessinsider.com/stocks/abt-stock',
 'https://markets.businessinsider.com/stocks/abbv-stock',
 'https://markets.businessinsider.com/stocks/acn-stock',
 'https://markets.businessinsider.com/stocks/adbe-stock',
 'https://markets.businessinsider.com/stocks/aes-stock',
 'https://markets.businessinsider.com/stocks/afl-stock',
 'https://markets.businessinsider.com/stocks/a-stock',
 'https://markets.businessinsider.com/stocks/apd-stock',
 'https://markets.businessinsider.com/stocks/abnb-stock',
 'https://markets.businessinsider.com/stocks/akam-stock',
 'https://markets.businessinsider.com/stocks/alb-stock',
 'https://markets.businessinsider.com/stocks/are-stock',
 'https://markets.businessinsider.com/stocks/algn-stock',
 'https://markets.businessinsider.com/stocks/alle-stock',
 'https://markets.businessinsider.com/stocks/lnt-stock',
 'https://markets.businessi

In [133]:
growth

[90.53,
 -21.3,
 15.39,
 16.83,
 -8.55,
 -16.91,
 -32.43,
 31.65,
 -13.48,
 28.17,
 -12.99,
 -24.82,
 -37.95,
 -17.32,
 -42.66,
 -1.5,
 29.52,
 27.08,
 31.67,
 32.01,
 34.98,
 20.09,
 9.46,
 -51.73,
 35.02,
 23.2,
 30.09,
 8.63,
 1.3,
 14.55,
 22.53,
 2.59,
 14.45,
 17.11,
 19.7,
 -2.41,
 27.43,
 -39.47,
 39.39,
 -26.79,
 -15.42,
 37.07,
 13.76,
 52.39,
 28.6,
 6.43,
 27.35,
 14.55,
 21.84,
 -14.58,
 38.48,
 -19.03,
 19.4,
 55.18,
 -18.21,
 -4.87,
 23.4,
 -3.43,
 -33.92,
 -16.71,
 23.31,
 -18.83,
 43.45,
 -8.61,
 7.22,
 55.31,
 12.06,
 41.91,
 19.44,
 -36.11,
 36.06,
 -21.73,
 -25.91,
 36.99,
 15.19,
 -1.84,
 39.54,
 12.39,
 0.85,
 13.6,
 54.17,
 -30.18,
 -64.14,
 7.18,
 -23.96,
 19.51,
 -6.46,
 -36.4,
 12.69,
 36.94,
 1.67,
 -0.38,
 14.55,
 8.02,
 21.56,
 31.22,
 30.08,
 27.35,
 29.14,
 -0.51,
 15.84,
 20.03,
 17.68,
 9.91,
 5.24,
 -15.63,
 -9.16,
 -20.64,
 11.52,
 -29.88,
 -0.38,
 43.37,
 13.2,
 -11.26,
 35.5,
 -0.68,
 -15.56,
 -18.82,
 27.02,
 -11.29,
 -14.82,
 -16.74,
 10.68,
 6.08

In [171]:
usd_cur = requests.get('https://www.cbr.ru/scripts/XML_daily.asp').text.find('./Valute[CharCode="USD"]/Value')
print(usd_cur)

-1


In [165]:
#Ищем стоимость компании, код и P/E
prices = []
codes = []
pe_ratios = []

for url in company_links: 
    response = requests.get(url)
    bs = BeautifulSoup(response.text, "html.parser")
    
    price = bs.find(name = "span", attrs = {"class": "price-section__current-value"}).text
    prices.append(float(price.replace(',','')))

    code = bs.find(name = "span", attrs = {"class": "price-section__category"}).text
    codes.append(code[-4:])

    right_data_items= bs.find_all(name = "div", attrs = {"class": "snapshot__data-item padding-right--zero"})
    for data_item in right_data_items:
        if data_item.find(name ="div", attrs = {"class": "snapshot__header"}, string ="P/E Ratio") is not None:
            pe_ratios.append(float(data_item.text.replace("P/E Ratio","")))


KeyboardInterrupt: 

In [ ]:
#Ищем и высчитываем разницу 52 Week Low и 52 Week High
profits = []

for url in company_links: 
    response = requests.get(url)
    bs = BeautifulSoup(response.text, "html.parser")
    
    small_data_items = bs.find_all(name = "div", attrs = {"class": "snapshot__data-item snapshot__data-item--small"})
    if len(small_data_items) == 0:
        ft_week_low = 0
    else:
        ft_week_low = float(small_data_items[len(small_data_items)-1].text.replace("52 Week Low","").replace(',',''))
    
    small_data_items_right = bs.find_all(name = "div", attrs = {"class": "snapshot__data-item snapshot__data-item--small snapshot__data-item--right"})
    if len(small_data_items_right) == 0:
        ft_week_high = 0
    else:
        ft_week_high = float(small_data_items_right[len(small_data_items_right)-1].text.replace("52 Week High","").replace(',',''))
    
    profits.append(ft_week_high - ft_week_low)

In [32]:
#текущая стоимость в рублях
price = bs.find(name = "span", attrs = {"class": "price-section__current-value"}).text
print(float(price))

1462.4


In [69]:
#код
code = bs.find(name = "span", attrs = {"class": "price-section__category"}).text
print(code[-4:])

MMM 


In [70]:
# P/E компании
right_data_items= bs.find_all(name = "div", attrs = {"class": "snapshot__data-item padding-right--zero"})
for data_item in right_data_items:
    if data_item.find(name ="div", attrs = {"class": "snapshot__header"}, string ="P/E Ratio") is not None:
        print(float(data_item.text.replace("P/E Ratio","")))

17.09


In [156]:
response = requests.get('https://markets.businessinsider.com/stocks/aapl-stock')
bs = BeautifulSoup(response.text, "html.parser")

In [163]:
#Находим цены 52 Week Low и 52 Week High

small_data_items = bs.find_all(name = "div", attrs = {"class": "snapshot__data-item snapshot__data-item--small"})
if len(small_data_items) == 0:
    ft_week_low = 0
else:
    ft_week_low = float(small_data_items[len(small_data_items)-1].text.replace("52 Week Low",""))

small_data_items_right = bs.find_all(name = "div", attrs = {"class": "snapshot__data-item snapshot__data-item--small snapshot__data-item--right"})
if len(small_data_items_right) == 0:
    ft_week_high = 0
else:
    ft_week_high = float(small_data_items_right[len(small_data_items_right)-1].text.replace("52 Week High",""))    
#     #print(small_data_items.text.replace("52 Week Low",""))
#     if small_data_items is not None:
#         for data_item in small_data_items:
#             print
#             ft_week_low = data_item.find(name ="div", attrs = {"class": "snapshot__header"}, string ="52 Week Low").text
#             ft_week_low = float(ft_week_low.text.replace("52 Week Low",""))
#     else:
#         ft_week_low = 0
#     
#     small_data_items_right = bs.find(name = "div", attrs = {"class": "snapshot__data-item snapshot__data-item--small snapshot__data-item--right"})
#     #print(small_data_items_right.text.replace("52 Week High",""))
#     if small_data_items_right is not None:
#         for data_item in small_data_items_right:
#             ft_week_high = data_item.find(name ="div", attrs = {"class": "snapshot__header"}, string ="52 Week High").text
#             ft_week_high = float(ft_week_high.text.replace("52 Week High",""))
#     else:
#         ft_week_high = 0
#     
# #print(ft_week_high - ft_week_low)

<div class="snapshot__data-item snapshot__data-item--small">
						164.08
						<div class="snapshot__header">52 Week Low</div>
</div>
